In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K

from PIL import Image 

In [ ]:
# Original width: 600
# IM_WIDTH = 400 # Keep width = 400 for InceptionV4
# IM_WIDTH = 300 # Keep width = 300 for ResNet50
IM_WIDTH = 224 # Keep width = 224 for InceptionV3

# Original height: 450
# IM_HEIGHT = 300 # Keep height = 300 for InceptionV4
# IM_HEIGHT = 225 # Keep height = 225 for ResNet50
IM_HEIGHT = 168 # Keep height = 168 for InceptionV3

BATCH_SIZE = 16

global_mean = np.load('/home/ubuntu/Data/Skin/nv_non_nv/mean_image_nv_non_nv.npz')['image'].astype(np.uint8)
mean_image = Image.fromarray(global_mean).resize((IM_WIDTH, IM_HEIGHT))
global_mean = np.asarray(mean_image, dtype=np.float32)

In [ ]:
def preprocess_image(image):
    return image - global_mean

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import compute_class_weight

train_img = ImageDataGenerator(preprocessing_function=preprocess_image, rescale=1./255)

img_gen = ImageDataGenerator(preprocessing_function=preprocess_image, rescale=1./255)

train_gen = train_img.flow_from_directory('/home/ubuntu/Data/Skin/nv_non_nv/train', 
                                        target_size=(IM_HEIGHT, IM_WIDTH), 
                                        class_mode='binary', 
                                        shuffle=True, 
                                        batch_size=BATCH_SIZE, 
                                        classes=['NON_NV', 'NV']
                                       )
print('Training Set Generator done.\n')
val_gen = img_gen.flow_from_directory('/home/ubuntu/Data/Skin/nv_non_nv/val', 
                                        target_size=(IM_HEIGHT, IM_WIDTH), 
                                        class_mode='binary', 
                                        shuffle=True, 
                                        batch_size=BATCH_SIZE, 
                                        classes=['NON_NV', 'NV']
                                     )
print('Validation Set Generator done.\n')
test_gen = img_gen.flow_from_directory('/home/ubuntu/Data/Skin/nv_non_nv/test', 
                                        target_size=(IM_HEIGHT, IM_WIDTH), 
                                        class_mode='binary', 
                                        shuffle=False, 
                                        batch_size=BATCH_SIZE, 
                                        classes=['NON_NV', 'NV']
                                      )
print('Test Set Generator done.\n')
y_train = train_gen.classes
weights = compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {i: weights[i] for i in range(len(weights))}

print(class_weights)
print(train_gen.class_indices)

In [ ]:
num_hidden_units = 1024
model_name = 'inceptionv3'
# model_name = 'resnet50'
# model_name = 'inceptionv4'
# model_name = 'vgg16'
# model_name = 'densenet'
training_set = 'full_train'

In [ ]:
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.resnet50 import ResNet50
# from keras.applications.vgg16 import VGG16
import sys
sys.path.append('/home/ubuntu/Notebooks/Models/')
# import inception_v4
from keras import Model
from keras.layers import (Dense, Activation, Dropout, 
                          BatchNormalization, Flatten, GlobalAveragePooling2D)

inc = InceptionV3(include_top=False, input_shape=(IM_HEIGHT, IM_WIDTH, 3), weights='imagenet')
# inc = ResNet50(include_top=False, input_shape=(IM_HEIGHT, IM_WIDTH, 3), weights='imagenet')
# inc = VGG16(include_top=False, input_shape=(IM_HEIGHT, IM_WIDTH, 3), weights='imagenet')
# inc = inception_v4.create_model(weights=None, include_top=False, input_shape=(IM_HEIGHT, IM_WIDTH, 3))
# inc.load_weights('/home/ubuntu/Notebooks/Models/inception-v4_weights_tf_dim_ordering_tf_kernels_notop.h5')
x = inc.output
# x = GlobalAveragePooling2D()(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(num_hidden_units)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.1)(x)
out = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inc.input, outputs=out)

for layer in inc.layers:
    layer.trainable = False
    
decision_weights = model.layers[-1].get_weights()
print(decision_weights[0].shape, decision_weights[1].shape)
decision_weights[1] -= 0.7
model.layers[-1].set_weights(decision_weights)
print(model.layers[-1].get_weights()[1])
filename = 'final_nv_non_nv_' + model_name + '_' + str(num_hidden_units) + '_' + training_set + '.h5'
print(filename)

In [ ]:
print(model.summary())

In [ ]:
def f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0., 1.)))
    precision = true_positives / (predicted_positives + K.epsilon())
    
    possible_positives = K.sum(K.round(K.clip(y_true, 0., 1.)))
    recall = true_positives / (possible_positives + K.epsilon())
    return ( 2*(precision * recall) / (precision + recall + K.epsilon()) )

In [ ]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0., 1.)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0., 1.)))
    return (true_positives / (predicted_positives + K.epsilon()))

In [ ]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0., 1.)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0., 1.)))
    return (true_positives / (possible_positives + K.epsilon()))

In [ ]:
from keras.optimizers import Adam, RMSprop
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])
# model.compile(loss='hinge', optimizer=Adam(1e-5), metrics=[svm_binary_accuracy])

_ = model.fit_generator(train_gen, 
                        steps_per_epoch=train_gen.samples//BATCH_SIZE + 1, 
                        validation_data=val_gen, 
                        validation_steps=val_gen.samples//BATCH_SIZE + 1, 
                        shuffle=True, 
                        epochs=1,
                        class_weight=class_weights)

In [ ]:
def reset_batchnorm(final_model):
    for i, layer in enumerate(final_model.layers):
        if type(layer) is BatchNormalization:
            input_shape = layer.input_shape
            gamma = np.ones((input_shape[-1],))
            beta = np.zeros((input_shape[-1],))
            moving_mean = np.zeros((input_shape[-1],))
            moving_variance = np.ones((input_shape[-1],))
            config = layer.get_config()
            if config['scale'] and config['center']:
                layer.set_weights([gamma, beta, moving_mean, moving_variance])
            elif config['scale'] and not config['center']:
                layer.set_weights([gamma, moving_mean, moving_variance])
            elif not config['scale'] and config['center']:
                layer.set_weights([beta, moving_mean, moving_variance])
#             print('Weights Changed')
        
    return final_model

In [ ]:
from keras.optimizers import Adam, SGD

for layer in model.layers:
    layer.trainable = True

# model = reset_batchnorm(model)
# print('All BatchNorm layers were reset')

optimizer = Adam(lr=1e-4, decay=5e-4)

model.compile(loss='binary_crossentropy', optimizer=optimizer, 
              metrics=['accuracy', precision, recall, f1])
# model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-4, decay=5e-2), metrics=['accuracy', precision, recall, f1])

print('Model compiled')
initial_epoch=0
num_epochs=20

In [ ]:
# Recompiling model AFTER running for x epochs.
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-7, decay=1e-3), metrics=['accuracy', precision, recall, f1])
print('Model re-compiled')
initial_epoch=13

filename = filename[:-3] + '_2.h5'
print(filename)


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# reducer = step_decay_schedule(initial_lr=1e-5, step_size=train_gen.samples//BATCH_SIZE)

# plateau_watcher = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

checkpoint = ModelCheckpoint(filename, 
                             save_best_only=True, 
                             mode='max',
                             monitor='val_f1', 
                             verbose=1)

stopper = EarlyStopping(monitor='val_loss', mode='min', patience=4)

h = model.fit_generator(train_gen, 
                        steps_per_epoch=train_gen.samples//BATCH_SIZE + 1, 
                        validation_data=val_gen, 
                        validation_steps=val_gen.samples//BATCH_SIZE + 1, 
                        shuffle=True, 
                        epochs=num_epochs, 
                        class_weight=class_weights, 
                        callbacks=[checkpoint, stopper])

fig = plt.figure()
plt.plot(h.history['loss'], 'r-')
plt.plot(h.history['val_loss'], 'b-')
plt.title('Loss plot')
plt.show()

fig = plt.figure()
plt.plot(h.history['f1'], 'r-')
plt.plot(h.history['val_f1'], 'b-')
plt.title('Loss plot')

plt.show()

In [ ]:
from keras.models import load_model
m = load_model(filename, compile=True, custom_objects={'f1':f1, 'precision':precision, 'recall':recall})
print('Best model loaded')

In [ ]:
pre_gen = img_gen.flow_from_directory('/home/ubuntu/Data/Skin/nv_non_nv/test', 
                                        target_size=(IM_HEIGHT, IM_WIDTH), 
                                        class_mode='binary', 
                                        shuffle=False, 
                                        batch_size=BATCH_SIZE,
                                      )

y_pred = model.predict_generator(pre_gen, steps=pre_gen.samples//BATCH_SIZE + 1, verbose=1)

In [ ]:
y_true = test_gen.classes

y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0

In [ ]:
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_true=y_true.ravel(), y_pred=y_pred.ravel()))

print(classification_report(y_true=y_true.ravel(), y_pred=y_pred.ravel()))

fpr, tpr, threshold = roc_curve(y_true, y_pred)
model_auc = auc(fpr, tpr)

print('AUC: {}'.format(model_auc))